# Poisson solution with FFT

## Problem

As a subroutine for the main algorithm. We need the ability to solve Poisson differential equations. In other words, equations of the form

$$
\begin{align*}
\Delta \phi(x, y, z) &= f(x, y, z), \forall (x, y, z) \in [0, L_x] \times [0, L_y] \times [0, L_z] \\
\phi (0, y) &= \phi (L_x, y), \phi (x, 0) = \phi (x, L_y) \\
\phi_x (0, y) &= \phi_x (L_x, y), \phi_y (x, 0) = \phi_y (x, L_y) \\
\end{align*}
$$,

a Poisson equation with periodic boundary constraints.  
To simplify the problem without much loss for generality, we set $L_x = L_y = L_z = 1$. Therefore, the domain $\Omega = [0, 1]^3$.

## Solution

The textbook way of solving these types of differential equations is to consider a Fourier series expansion forms of the functions $\phi$ and $f$.



## Discretization

However, as we're dealing with discrete data points, we only have access to the function values at specific points. 



In [1]:
import matplotlib as plt